O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

## Inicialização & Vizualização

In [1]:
!pip install lightgbm
!pip install catboost
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('/datasets/car_data.csv')
df

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


* Muitas alterações devem ser feitas, desde mudar o dtype, preencher nans e dar um lowercase nas primeiras letras dos nomes das colunas.

## Preparação de Dados

In [6]:
df.columns = df.columns.str.lower()

* Colunas todas com letras minúsculas

### Coluna model

In [7]:
(df['model'].isna().sum()/len(df))*100

5.560587974681758

* A porcentagem de modelos desconhecidos é baixa, somente 5%, então irei substituir por 'unknown'

In [8]:
df['model'] = df['model'].fillna('unknown')
df['model'].isna().sum()

0

### Coluna vehicletype

In [9]:
(df['vehicletype'].isna().sum()/len(df))*100

10.57936783409384

* 10% é bastante para substituir pela moda, por isso irei criar uma coluna com a moda de vehicletype agrupada por cada modelo de carro e substituir pelos nans.

In [10]:
df['modelo_freq'] = df.groupby('model')['vehicletype'].transform(lambda x: x.mode()[0])

In [11]:
df['vehicletype'] = df['vehicletype'].fillna(df['modelo_freq'])
df['vehicletype'].isna().sum()

0

### Coluna gearbox

In [12]:
(df['gearbox'].isna().sum()/len(df))*100

5.596708515699737

* Representa apenas 5% então substituirei pela moda

In [13]:
df['gearbox'] = df['gearbox'].fillna(df['gearbox'].mode()[0])
df['gearbox'].isna().sum()

0

### Coluna fueltype

In [14]:
(df['fueltype'].isna().sum()/len(df))*100

9.282696849893755

* 9% é bastante, irei fazer o mesmo que fiz com a coluna vehicletype

In [15]:
df['fuel_freq'] = df.groupby('model')['fueltype'].transform(lambda x: x.mode()[0])

In [16]:
df['fueltype'] = df['fueltype'].fillna(df['fuel_freq'])
df['fueltype'].isna().sum()

0

### Coluna notrepaired

In [17]:
(df['notrepaired'].isna().sum()/len(df))*100

20.079070121822166

In [18]:
df['notrepaired'].value_counts()

no     247161
yes     36054
Name: notrepaired, dtype: int64

* Por mais que 20% seja um valor alto, não podemos assumir que o carro está reparado ou não então para proteger a empresa já que o carro reparado provavelmente aumentaria considerávelmente o valor do carro, deixaremos os nan no. E também por ser uma classificação binária irei substituir os no por 0 e yes por 1.

In [19]:
df['notrepaired'] = df['notrepaired'].fillna('no')
df['notrepaired'].isna().sum()

0

In [20]:
df['notrepaired'] = df['notrepaired'].replace('no', 0)
df['notrepaired'] = df['notrepaired'].replace('yes', 1)

In [21]:
df['notrepaired'].value_counts()

0    318315
1     36054
Name: notrepaired, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        354369 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            354369 non-null  object
 5   power              354369 non-null  int64 
 6   model              354369 non-null  object
 7   mileage            354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           354369 non-null  object
 10  brand              354369 non-null  object
 11  notrepaired        354369 non-null  int64 
 12  datecreated        354369 non-null  object
 13  numberofpictures   354369 non-null  int64 
 14  postalcode         354369 non-null  int64 
 15  lastseen           354369 non-null  object
 16  modelo_freq        3

### Colunas datecrawled, datecreated, lastseen
* todas transformadas em dtype de data

In [23]:
df['datecrawled'] = pd.to_datetime(df['datecrawled'], format='%d/%m/%Y %H:%M')

In [24]:
df['datecreated'] = pd.to_datetime(df['datecreated'], format='%d/%m/%Y %H:%M')

In [25]:
df['lastseen'] = pd.to_datetime(df['lastseen'], format='%d/%m/%Y %H:%M')

In [26]:
df

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen,modelo_freq,fuel_freq
0,2016-03-24 11:52:00,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,0,2016-03-24,0,70435,2016-04-07 03:16:00,sedan,petrol
1,2016-03-24 10:58:00,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,1,2016-03-24,0,66954,2016-04-07 01:46:00,sedan,petrol
2,2016-03-14 12:52:00,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,0,2016-03-14,0,90480,2016-04-05 12:47:00,suv,gasoline
3,2016-03-17 16:54:00,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,0,2016-03-17,0,91074,2016-03-17 17:40:00,sedan,petrol
4,2016-03-31 17:25:00,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,0,2016-03-31,0,60437,2016-04-06 10:17:00,small,petrol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:00,0,sedan,2005,manual,0,colt,150000,7,petrol,mitsubishi,1,2016-03-21,0,2694,2016-03-21 10:42:00,sedan,petrol
354365,2016-03-14 17:48:00,2200,sedan,2005,manual,0,unknown,20000,1,petrol,sonstige_autos,0,2016-03-14,0,39576,2016-04-06 00:46:00,sedan,petrol
354366,2016-03-05 19:56:00,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,0,2016-03-05,0,26135,2016-03-11 18:17:00,small,petrol
354367,2016-03-19 18:57:00,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,0,2016-03-19,0,87439,2016-04-07 07:15:00,bus,gasoline


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   datecrawled        354369 non-null  datetime64[ns]
 1   price              354369 non-null  int64         
 2   vehicletype        354369 non-null  object        
 3   registrationyear   354369 non-null  int64         
 4   gearbox            354369 non-null  object        
 5   power              354369 non-null  int64         
 6   model              354369 non-null  object        
 7   mileage            354369 non-null  int64         
 8   registrationmonth  354369 non-null  int64         
 9   fueltype           354369 non-null  object        
 10  brand              354369 non-null  object        
 11  notrepaired        354369 non-null  int64         
 12  datecreated        354369 non-null  datetime64[ns]
 13  numberofpictures   354369 non-null  int64   

## Treinamento do modelo


### Features & Targets

In [28]:
df_encoded = pd.get_dummies(df, columns=['vehicletype', 'gearbox', 'model', 'fueltype', 'brand'])

In [29]:
X = df_encoded.drop(['price', 'datecrawled', 'datecreated', 'lastseen', 'modelo_freq', 'fuel_freq'], axis=1)
y= df_encoded['price']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=12345)

### Random Forest

In [31]:
%%time
model1 = RandomForestRegressor(random_state=12345)
scores = cross_val_score(model1, X, y, cv=2, scoring='neg_root_mean_squared_error') # cv=2 pois estava demorando demais com cv = 5
avg_score = -scores.mean()
print(f'RandomForestRegressor - Média do Root Mean Squared Error: {avg_score}')

RandomForestRegressor - Média do Root Mean Squared Error: 1763.279578016011
CPU times: user 16min 52s, sys: 4.1 s, total: 16min 56s
Wall time: 16min 57s


### LightGBM

In [32]:
modelolgbmsk1 = lgb.LGBMRegressor()
modelolgbmsk2 = lgb.LGBMRegressor(boosting_type = 'dart')
modelolgbmsk3 = lgb.LGBMRegressor(boosting_type = 'goss')

In [33]:
%%time
scores = cross_val_score(modelolgbmsk1, X, y, cv=5, scoring='neg_root_mean_squared_error')
avg_score = -scores.mean()
print(f'gbdt - Média do Root Mean Squared Error: {avg_score}')

gbdt - Média do Root Mean Squared Error: 1852.3989469506203
CPU times: user 49.4 s, sys: 1.59 s, total: 51 s
Wall time: 51.2 s


In [34]:
%%time
scores = cross_val_score(modelolgbmsk2, X, y, cv=5, scoring='neg_root_mean_squared_error')
avg_score = -scores.mean()
print(f'dart - Média do Root Mean Squared Error: {avg_score}')

dart - Média do Root Mean Squared Error: 2095.052290055777
CPU times: user 2min 15s, sys: 1.88 s, total: 2min 17s
Wall time: 2min 17s


In [35]:
%%time
scores = cross_val_score(modelolgbmsk3, X, y, cv=5, scoring='neg_root_mean_squared_error')
avg_score = -scores.mean()
print(f'goss - Média do Root Mean Squared Error: {avg_score}')

goss - Média do Root Mean Squared Error: 1850.083032227376
CPU times: user 1min 13s, sys: 1.57 s, total: 1min 14s
Wall time: 1min 15s


### CatBoost

In [36]:
%%time
modelocatb= CatBoostRegressor()
scores = cross_val_score(modelocatb, X, y, cv=5, scoring='neg_root_mean_squared_error')
avg_score = -scores.mean()
print(f'Cat - Média do Root Mean Squared Error: {avg_score}')

Learning rate set to 0.099927
0:	learn: 4234.5488545	total: 117ms	remaining: 1m 56s
1:	learn: 3993.0717232	total: 191ms	remaining: 1m 35s
2:	learn: 3786.0273310	total: 257ms	remaining: 1m 25s
3:	learn: 3593.7310142	total: 323ms	remaining: 1m 20s
4:	learn: 3427.6696827	total: 389ms	remaining: 1m 17s
5:	learn: 3288.3080196	total: 456ms	remaining: 1m 15s
6:	learn: 3158.2586157	total: 523ms	remaining: 1m 14s
7:	learn: 3052.0419702	total: 593ms	remaining: 1m 13s
8:	learn: 2953.4526545	total: 660ms	remaining: 1m 12s
9:	learn: 2869.3427697	total: 730ms	remaining: 1m 12s
10:	learn: 2791.7893204	total: 797ms	remaining: 1m 11s
11:	learn: 2726.6318108	total: 862ms	remaining: 1m 10s
12:	learn: 2667.9488386	total: 923ms	remaining: 1m 10s
13:	learn: 2616.4910492	total: 985ms	remaining: 1m 9s
14:	learn: 2573.1254997	total: 1.05s	remaining: 1m 8s
15:	learn: 2532.7643124	total: 1.11s	remaining: 1m 8s
16:	learn: 2495.6536747	total: 1.18s	remaining: 1m 8s
17:	learn: 2466.1311412	total: 1.24s	remaining: 1

###  XGBoost

In [37]:
%%time
modeloxgb= xgb.XGBRegressor()
scores = cross_val_score(modeloxgb, X, y, cv=5, scoring='neg_root_mean_squared_error')
avg_score = -scores.mean()
print(f'xgb - Média do Root Mean Squared Error: {avg_score}')

xgb - Média do Root Mean Squared Error: 1807.0582720963055
CPU times: user 47min 23s, sys: 6.89 s, total: 47min 30s
Wall time: 47min 32s


## Análise do modelo

* Catboost foi claramente o melhor modelo, pelo tempo de resposta e pelo Root Mean Squared Error em que obteve o melhor resultado.

* Por mais que lightgbm e catboost não precisam de OHE decidi usar em todos.

* Irei realizar a otimização dos hiperparâmetros do catboost e treinar de fato o modelo para ter previsões.

(Comentário para o revisor: Usei os parâmetros do lightgbm para conseguir usar features categoricas sem o ohe porém não deu certo. Usei este hiperparâmetro - categorical_feature=categorical_features - onde categorical_features era uma lista das colunas categoricas. Acredito que usei da maneira adequada mas não funcionou por alguma rasão, então optei pelo OHE.)

In [38]:
#%%time
#param_grid = {
    #'iterations': [100, 200, 300],
    #'learning_rate': [0.01, 0.1, 0.2],
    #'depth': [4, 6, 8, 10],
    #'l2_leaf_reg': [1, 3, 5, 7, 9]}

#grid_search = GridSearchCV(estimator=modelocatb, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error')
#grid_search.fit(X_train, y_train)

#best_params = grid_search.best_params_
#best_score = grid_search.best_score_
#print("Melhores hiperparâmetros:", best_params)
#print("Melhor pontuação:", best_score)


* Tudo está como comentário, pois demorei quase 4hrs para saber quais eram os melhores hiperparâmetros

In [39]:
%%time
melhor_modelo = CatBoostRegressor(max_depth= 10, iterations=300, l2_leaf_reg= 1, learning_rate= 0.2)
melhor_modelo.fit(X_train, y_train)
y_pred = melhor_modelo.predict(X_test)


0:	learn: 3918.2587967	total: 159ms	remaining: 47.5s
1:	learn: 3462.3524941	total: 327ms	remaining: 48.7s
2:	learn: 3113.4994641	total: 482ms	remaining: 47.7s
3:	learn: 2855.9878584	total: 628ms	remaining: 46.5s
4:	learn: 2661.1119786	total: 785ms	remaining: 46.3s
5:	learn: 2514.0804035	total: 957ms	remaining: 46.9s
6:	learn: 2404.2112249	total: 1.13s	remaining: 47.2s
7:	learn: 2321.9164539	total: 1.29s	remaining: 47.2s
8:	learn: 2250.5070608	total: 1.46s	remaining: 47.2s
9:	learn: 2192.4949900	total: 1.62s	remaining: 47.1s
10:	learn: 2150.4315089	total: 1.79s	remaining: 47s
11:	learn: 2114.8135631	total: 1.96s	remaining: 47s
12:	learn: 2087.0043337	total: 2.11s	remaining: 46.7s
13:	learn: 2059.4885685	total: 2.28s	remaining: 46.6s
14:	learn: 2040.2859314	total: 2.44s	remaining: 46.4s
15:	learn: 2025.3201122	total: 2.61s	remaining: 46.3s
16:	learn: 2010.0338172	total: 2.77s	remaining: 46.1s
17:	learn: 1998.6117436	total: 2.93s	remaining: 46s
18:	learn: 1985.4468923	total: 3.09s	remaini

In [40]:
%%time
cross_val_score(melhor_modelo,X_train,y_train, scoring='neg_root_mean_squared_error').mean()
#repeti este codigo apenas para mostrar o RMSE

0:	learn: 3918.8075317	total: 138ms	remaining: 41.3s
1:	learn: 3464.2253081	total: 246ms	remaining: 36.7s
2:	learn: 3112.7372542	total: 371ms	remaining: 36.7s
3:	learn: 2854.1990353	total: 493ms	remaining: 36.5s
4:	learn: 2666.4347144	total: 628ms	remaining: 37.1s
5:	learn: 2519.5482621	total: 759ms	remaining: 37.2s
6:	learn: 2409.1616372	total: 895ms	remaining: 37.5s
7:	learn: 2317.3275282	total: 1.01s	remaining: 37s
8:	learn: 2249.8245478	total: 1.15s	remaining: 37.1s
9:	learn: 2196.5474827	total: 1.28s	remaining: 37.2s
10:	learn: 2148.7525057	total: 1.41s	remaining: 37.2s
11:	learn: 2115.4048804	total: 1.54s	remaining: 36.9s
12:	learn: 2083.5342287	total: 1.67s	remaining: 37s
13:	learn: 2058.4949959	total: 1.81s	remaining: 37s
14:	learn: 2038.6172041	total: 1.95s	remaining: 37s
15:	learn: 2020.9149934	total: 2.07s	remaining: 36.7s
16:	learn: 2004.4513621	total: 2.2s	remaining: 36.7s
17:	learn: 1994.3038534	total: 2.34s	remaining: 36.6s
18:	learn: 1983.5796824	total: 2.47s	remaining:

-1747.204790443197

In [41]:
%%time
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')
#o melhor RMSE

RMSE: 1739.5135540945873
CPU times: user 3.44 ms, sys: 0 ns, total: 3.44 ms
Wall time: 2.22 ms


**Conclusão**

* O melhor modelo foi o catboost com os hiperpâmetros:  {'depth': 10, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.2}. Obtendo uma pontuação de -1747.204790443197 em negative root mean squared error.
* O que significa que o conjunto de treinamento ficou 1747.204790443197 Euros distante do preço ideal. E a predição junto com o conjunto de teste ficou com 1739.5135540945873 Euros distante do preço ideal obtendo um desempenho ainda melhor e tudo isso em apenas 51.2 segundos para o fit do melhor modelo o que é excelente.
* Exemplificando, se voçê botasse seu carro neste projeto ele ficaria aproximadamente 1739.5135540945873 Euros distante do preço ideal. Considerando o range de preços deste dataframe de até 20000 ficar numa variância de menos de 2000 é um boim desempenho, pois equivale menos de 10%.